### Load hourly weather data and derive daily date field  
Read the raw hourly weather dataset from disk, parse the timestamp column into a proper datetime format, and extract the calendar date to support daily aggregations and downstream analyses.

In [7]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import warnings

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)


RAW_PATH = Path("../data/raw/hourly_weather_data.csv")
OUT_DIR  = Path("../data/processed/")

df = pd.read_csv(RAW_PATH)

# parse timestamp (adjust column name if yours is "Date/Time (LST)")
df["Date/Time (LST)"] = pd.to_datetime(df["Date/Time (LST)"], errors="coerce")
df["date"] = df["Date/Time (LST)"].dt.date

df.head()

,Date/Time (LST),Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Time (LST),Flag,...,Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,station_id,date
0,2025-01-01 00:00:00,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,1,1,00:00,NaN,...,NaN,95.56,NaN,NaN,NaN,NaN,NaN,NaN,42967,2025-01-01
1,2025-01-01 01:00:00,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,1,1,01:00,NaN,...,NaN,95.48,NaN,NaN,NaN,NaN,NaN,NaN,42967,2025-01-01
2,2025-01-01 02:00:00,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,1,1,02:00,NaN,...,NaN,95.48,NaN,NaN,NaN,-1.0,NaN,NaN,42967,2025-01-01
3,2025-01-01 03:00:00,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,1,1,03:00,NaN,...,NaN,95.49,NaN,NaN,NaN,NaN,NaN,NaN,42967,2025-01-01
4,2025-01-01 04:00:00,-78.27,45.53,ALGONQUIN PARK EAST GATE,6080192,2025,1,1,04:00,NaN,...,NaN,95.48,NaN,NaN,NaN,-1.0,NaN,NaN,42967,2025-01-01


### Compute daily weather summaries from hourly observations  
Aggregate hourly weather measurements to the daily level for each station, preserving static station metadata and calculating daily mean temperature, maximum relative humidity, and total precipitation.

In [8]:
# Daily aggregates
daily = df.groupby(["Station Name", "date"]).agg(
    **{
        "Longitude (x)": ("Longitude (x)", "first"),
        "Latitude (y)": ("Latitude (y)", "first"),
        "Climate ID": ("Climate ID", "first"),
        "station_id": ("station_id", "first"),
        "mean_temp": ("Temp (°C)", "mean"),
        "max_rel_hum": ("Rel Hum (%)", "max"),
        "precip_mm_sum": ("Precip. Amount (mm)", "sum"),
    }
).reset_index()

daily.head()

,Station Name,date,Longitude (x),Latitude (y),Climate ID,station_id,mean_temp,max_rel_hum,precip_mm_sum
0,ALGONQUIN PARK EAST GATE,2023-01-01,-78.27,45.53,6080192,42967,-0.345833,98.0,0.2
1,ALGONQUIN PARK EAST GATE,2023-01-02,-78.27,45.53,6080192,42967,0.470833,98.0,0.4
2,ALGONQUIN PARK EAST GATE,2023-01-03,-78.27,45.53,6080192,42967,0.183333,97.0,0.7
3,ALGONQUIN PARK EAST GATE,2023-01-04,-78.27,45.53,6080192,42967,-0.408333,97.0,21.5
4,ALGONQUIN PARK EAST GATE,2023-01-05,-78.27,45.53,6080192,42967,-1.187500,98.0,8.1


### Calculate wet-hour metrics and merge into daily dataset

Identify hours with high relative humidity (≥ 90%) to compute daily wet-hour counts and mean wet-period temperature, then merge these metrics into the daily weather table and handle missing values.

In [9]:
# Wet-hours stats (Rel Hum >= 90)
wet = df[df["Rel Hum (%)"] >= 90].groupby(["Station Name", "date"]).agg(
    wet_hours_count=("Rel Hum (%)", "count"),
    wet_hours_temp=("Temp (°C)", "mean"),
).reset_index()


daily = daily.merge(wet, on=["Station Name", "date"], how="left")
daily["wet_hours_count"] = daily["wet_hours_count"].fillna(0)
daily["wet_hours_temp"]  = daily["wet_hours_temp"].fillna(daily["mean_temp"])

daily.head()

,Station Name,date,Longitude (x),Latitude (y),Climate ID,station_id,mean_temp,max_rel_hum,precip_mm_sum,wet_hours_count,wet_hours_temp
0,ALGONQUIN PARK EAST GATE,2023-01-01,-78.27,45.53,6080192,42967,-0.345833,98.0,0.2,18.0,-0.372222
1,ALGONQUIN PARK EAST GATE,2023-01-02,-78.27,45.53,6080192,42967,0.470833,98.0,0.4,17.0,0.047059
2,ALGONQUIN PARK EAST GATE,2023-01-03,-78.27,45.53,6080192,42967,0.183333,97.0,0.7,12.0,-0.625000
3,ALGONQUIN PARK EAST GATE,2023-01-04,-78.27,45.53,6080192,42967,-0.408333,97.0,21.5,24.0,-0.408333
4,ALGONQUIN PARK EAST GATE,2023-01-05,-78.27,45.53,6080192,42967,-1.187500,98.0,8.1,24.0,-1.187500


### Compute daily Disease Severity Values (DSV)  
Apply temperature- and wet-hour–dependent rules to calculate a daily disease severity value for each station and date, translating environmental conditions into discrete risk scores.

These calculations are based on potato late blight models shown in https://ipm.ucanr.edu/DISEASE/DATABASE/potatolateblight.html


In [10]:
def calculate_dsv(temp, wet_hours):
    """Calculate DSV based on temperature and total wet hours"""
    if wet_hours == 0 or pd.isna(temp):
        return 0

    if 7.2 <= temp <= 11.6:
        bins = [0, 15, 18, 21, 24, float('inf')]
    elif 11.7 <= temp <= 15.0:
        bins = [0, 12, 15, 18, 21, float('inf')]
    elif 15.1 <= temp <= 26.6:
        bins = [0, 9, 12, 15, 18, float('inf')]
    else:
        return 0

    sv_scale = [0, 1, 2, 3, 4]
    for i in range(5):
        if wet_hours <= bins[i + 1]:
            return sv_scale[i]
    return 0

daily["dsv"] = daily.apply(lambda r: calculate_dsv(r["wet_hours_temp"], r["wet_hours_count"]), axis=1)
daily.head()

,Station Name,date,Longitude (x),Latitude (y),Climate ID,station_id,mean_temp,max_rel_hum,precip_mm_sum,wet_hours_count,wet_hours_temp,dsv
0,ALGONQUIN PARK EAST GATE,2023-01-01,-78.27,45.53,6080192,42967,-0.345833,98.0,0.2,18.0,-0.372222,0
1,ALGONQUIN PARK EAST GATE,2023-01-02,-78.27,45.53,6080192,42967,0.470833,98.0,0.4,17.0,0.047059,0
2,ALGONQUIN PARK EAST GATE,2023-01-03,-78.27,45.53,6080192,42967,0.183333,97.0,0.7,12.0,-0.625000,0
3,ALGONQUIN PARK EAST GATE,2023-01-04,-78.27,45.53,6080192,42967,-0.408333,97.0,21.5,24.0,-0.408333,0
4,ALGONQUIN PARK EAST GATE,2023-01-05,-78.27,45.53,6080192,42967,-1.187500,98.0,8.1,24.0,-1.187500,0


### Assign daily disease risk categories from DSV values  
Convert daily disease severity values into categorical risk levels (LOW to VERY HIGH) to support interpretation, mapping, and decision-making.

In [11]:
def categorize_risk(dsv):
    if dsv <= 1: return "LOW"
    elif dsv <= 2: return "MODERATE"
    elif dsv <= 3: return "HIGH"
    else: return "VERY HIGH"

daily["disease_risk"] = daily["dsv"].apply(categorize_risk)

daily.head()

,Station Name,date,Longitude (x),Latitude (y),Climate ID,station_id,mean_temp,max_rel_hum,precip_mm_sum,wet_hours_count,wet_hours_temp,dsv,disease_risk
0,ALGONQUIN PARK EAST GATE,2023-01-01,-78.27,45.53,6080192,42967,-0.345833,98.0,0.2,18.0,-0.372222,0,LOW
1,ALGONQUIN PARK EAST GATE,2023-01-02,-78.27,45.53,6080192,42967,0.470833,98.0,0.4,17.0,0.047059,0,LOW
2,ALGONQUIN PARK EAST GATE,2023-01-03,-78.27,45.53,6080192,42967,0.183333,97.0,0.7,12.0,-0.625000,0,LOW
3,ALGONQUIN PARK EAST GATE,2023-01-04,-78.27,45.53,6080192,42967,-0.408333,97.0,21.5,24.0,-0.408333,0,LOW
4,ALGONQUIN PARK EAST GATE,2023-01-05,-78.27,45.53,6080192,42967,-1.187500,98.0,8.1,24.0,-1.187500,0,LOW


### Calculate cumulative DSVs and generate spray recommendation flags  
Accumulate daily disease severity values over time for each station, evaluate spray thresholds based on cumulative risk levels, and assign clear spray action flags and recommendations.

Action is recommended when cummulative DSV reaches the thresholds of 135, 180 and 225.

In [12]:
# ensure proper ordering
daily["date"] = pd.to_datetime(daily["date"])
daily = daily.sort_values(["Station Name", "date"])

# cumulative DSV per station
daily["dsv_total"] = daily.groupby("Station Name")["dsv"].cumsum()

# spray flags based on cumulative thresholds
daily["spray_1"] = daily["dsv_total"] >= 135
daily["spray_2"] = daily["dsv_total"] >= 180
daily["spray_3"] = daily["dsv_total"] >= 225

# optional: single spray recommendation column
def spray_recommendation(val):
    if val >= 225: return "SPRAY 3"
    elif val >= 180: return "SPRAY 2"
    elif val >= 135: return "SPRAY 1"
    else: return "NO SPRAY"

daily["spray_recommendation"] = daily["dsv_total"].apply(spray_recommendation)

daily.head()

,Station Name,date,Longitude (x),Latitude (y),Climate ID,station_id,mean_temp,max_rel_hum,precip_mm_sum,wet_hours_count,wet_hours_temp,dsv,disease_risk,dsv_total,spray_1,spray_2,spray_3,spray_recommendation
0,ALGONQUIN PARK EAST GATE,2023-01-01,-78.27,45.53,6080192,42967,-0.345833,98.0,0.2,18.0,-0.372222,0,LOW,0,False,False,False,NO SPRAY
1,ALGONQUIN PARK EAST GATE,2023-01-02,-78.27,45.53,6080192,42967,0.470833,98.0,0.4,17.0,0.047059,0,LOW,0,False,False,False,NO SPRAY
2,ALGONQUIN PARK EAST GATE,2023-01-03,-78.27,45.53,6080192,42967,0.183333,97.0,0.7,12.0,-0.625000,0,LOW,0,False,False,False,NO SPRAY
3,ALGONQUIN PARK EAST GATE,2023-01-04,-78.27,45.53,6080192,42967,-0.408333,97.0,21.5,24.0,-0.408333,0,LOW,0,False,False,False,NO SPRAY
4,ALGONQUIN PARK EAST GATE,2023-01-05,-78.27,45.53,6080192,42967,-1.187500,98.0,8.1,24.0,-1.187500,0,LOW,0,False,False,False,NO SPRAY


### Visualize station-level disease risk on an interactive Ontario map for a selected date  
Display all weather stations on an interactive map, with marker color indicating daily disease risk level and bubble size/transparency adjusted for clarity.

In [ ]:
# ===============================
# 👇 TYPE THE DATE YOU WANT HERE
# ===============================
MAP_DATE = "2025-08-19"   # YYYY-MM-DD
# ===============================

map_date = pd.to_datetime(MAP_DATE)

# filter to selected date
map_df = daily[daily["date"] == map_date].copy()

# one row per station (safety)
map_df = (
    map_df.sort_values("date")
          .groupby("Station Name", as_index=False)
          .tail(1)
)

risk_colors = {
    "LOW": "green",
    "MODERATE": "yellow",
    "HIGH": "orange",
    "VERY HIGH": "red",
}

fig = px.scatter_map(
    map_df,
    lat="Latitude (y)",
    lon="Longitude (x)",
    hover_name="Station Name",
    hover_data={
        "date": True,
        "dsv": True,
        "dsv_total": True,
        "Latitude (y)": False,
        "Longitude (x)": False
    },
    color="disease_risk",
    color_discrete_map=risk_colors,
    zoom=5,
    height=650,
)

fig.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.update_traces(
    marker=dict(
        size=18,
        opacity=0.5
    )
)

In [15]:
OUT_FIG = Path("../outputs")
OUT_FIG.mkdir(parents=True, exist_ok=True)

png_path = OUT_FIG / f"ontario_risk_map_{MAP_DATE}.png"
fig.write_image(str(png_path), scale=2)

png_path

WindowsPath('../outputs/ontario_risk_map_2025-08-19.png')